## RTP Data Visualizations

In [3]:
# get data for infographic
import pandas as pd
import pathlib
import os

# current working directory
local_path = pathlib.Path().absolute()

# get bonus_condit
# set data path as a subfolder of the current working directory TravelDemandModel\2022\
data_dir = local_path.parents[0] / 'data'
# folder to save processed data
out_dir  = local_path.parents[0] / 'data/processed_data'
df = pd.read_csv(data_dir / "Parcels_Forecast.csv")

In [ ]:
df.columns.to_list()

In [ ]:
df.Residential_Units.sum()

In [ ]:
# get percent of total Forecasted Residential Units by Location to Town Center
dfSummary = df.groupby(['LOCATION_TO_TOWNCENTER']).agg({'Residential_Units':'sum',
                                                        'FORECASTED_RESIDENTIAL_UNITS':'sum',
                                                        }).reset_index()

# get total forecasted residential units
dfSummary['Total_Residential_Units'] = dfSummary['FORECASTED_RESIDENTIAL_UNITS'] + dfSummary['Residential_Units']
# get percent of total forecasted residential units
dfSummary['Percent_FutureTotal_Residential_Units'] = (dfSummary['Total_Residential_Units'] / dfSummary['Total_Residential_Units'].sum() * 100).round(0)
dfSummary['Percent_CurrentTotal_Residential_Units'] =(dfSummary['Residential_Units'] / dfSummary['Residential_Units'].sum() * 100).round(0)

# filter dataframe to just perecnt fields
dfResSummary = dfSummary[['LOCATION_TO_TOWNCENTER','Percent_FutureTotal_Residential_Units','Percent_CurrentTotal_Residential_Units']]
dfResSummary


In [ ]:
# get percent of total Forecasted Tourist Units by Location to Town Center
dfSummary = df.groupby(['LOCATION_TO_TOWNCENTER']).agg({'TouristAccommodation_Units':'sum',
                                                        'FORECASTED_TOURIST_UNITS':'sum',
                                                        }).reset_index()

# get total forecasted tourist units
dfSummary['Total_Tourist_Units'] = dfSummary['FORECASTED_TOURIST_UNITS'] + dfSummary['TouristAccommodation_Units']
# get percent of total forecasted tourist units
dfSummary['Percent_FutureTotal_Tourist_Units'] = (dfSummary['Total_Tourist_Units'] / dfSummary['Total_Tourist_Units'].sum() * 100).round(0)
dfSummary['Percent_CurrentTotal_Tourist_Units'] =(dfSummary['TouristAccommodation_Units'] / dfSummary['TouristAccommodation_Units'].sum() * 100).round(0)
# filter dataframe to just perecnt fields
# dfTouristSummary = dfSummary[['LOCATION_TO_TOWNCENTER','Percent_FutureTotal_Tourist_Units','Percent_CurrentTotal_Tourist_Units']]
dfSummary

In [ ]:
dfSummary.TouristAccommodation_Units.sum()

In [ ]:
11039 + 698

In [ ]:
9645+698

In [ ]:
10343/11737

In [ ]:
# get percent of total Forecasted Commercial Floor Area by Location to Town Center
dfSummary = df.groupby(['LOCATION_TO_TOWNCENTER']).agg({'CommercialFloorArea_SqFt':'sum',
                                                        'FORECASTED_COMMERCIAL_SQFT':'sum',
                                                        }).reset_index()

# get total forecasted commercial floor area
dfSummary['Total_Commercial_SqFt'] = dfSummary['FORECASTED_COMMERCIAL_SQFT'] + dfSummary['CommercialFloorArea_SqFt']
# get percent of total forecasted commercial floor area
dfSummary['Percent_FutureTotal_Commercial_SqFt'] = (dfSummary['Total_Commercial_SqFt'] / dfSummary['Total_Commercial_SqFt'].sum() * 100).round(0)
dfSummary['Percent_CurrentTotal_Commercial_SqFt'] =(dfSummary['CommercialFloorArea_SqFt'] / dfSummary['CommercialFloorArea_SqFt'].sum() * 100).round(0)
# filter dataframe to just perecnt fields
# dfCommercialSummary = dfSummary[['LOCATION_TO_TOWNCENTER','Percent_FutureTotal_Commercial_SqFt','Percent_CurrentTotal_Commercial_SqFt']]
# dfCommercialSummary
dfSummary

In [ ]:
dfSummary.CommercialFloorArea_SqFt.sum()

### Forecast Formatted Table

In [ ]:
# get forecast data
forecast = pd.read_csv(data_dir / "Forecasts_Table1.csv")
# drop notes column 
forecast.drop(columns=['Notes'], inplace=True)
# change column names
forecast.rename(columns={'Change by 2050': 'Change(#)', 'Percent Change': 'Change(%)'}, inplace=True)

In [ ]:
GT(forecast).tab_header(title="Table 1. Forecast Data Summary").tab_spanner(
    label="", columns=['Category', 'Variable','Base Year 2022',  'Forecast 2050', 'Change(#)', 'Change(%)']).tab_stub(
        rowname_col='Variable', groupname_col='Category').tab_style(
            style=style.fill(color="aliceblue"), locations=loc.body()).save("forecast.jpeg")


### Transit Stacked Bar

In [ ]:
# get data for transit ridership
def get_data_transit():
    url = "https://www.laketahoeinfo.org/WebServices/GetTransitMonitoringData/CSV/e17aeb86-85e3-4260-83fd-a2b32501c476"

    dfTransit = pd.read_csv(url)
    dfTransit['Month'] = pd.to_datetime(dfTransit['Month'])
    dfTransit['Month'] = dfTransit['Month'].dt.strftime('%Y-%m')
    # filter out rows where RouteType is not Paratransit, Commuter, or Seasonal Fixed
    df = dfTransit.loc[~dfTransit['RouteType'].isin(['Paratransit', 'Commuter', 'Seasonal Fixed Route'])]
    # df = dfTransit.loc[dfTransit['RouteType'] != 'Paratransit']

    # replace transit operator values with abreviations
    df['TransitOperator'] = df['TransitOperator'].replace(
        ['Tahoe Transportation District',
       'Tahoe Truckee Area Regional Transit',
       'South Shore Transportation Management Association'],
       ["TTD", "TART", "SSTMA"])
    # route name = route type + transit operator
    df['RouteName'] = df['RouteType'] + ' - ' + df['TransitOperator']
    # group by RouteType, TransitOperator, and Month with sum of MonthlyRidership
    df = df.groupby(['RouteName', 'Month'])['MonthlyRidership'].sum().reset_index()
    # rename columns to Date, Name, Ridership
    df.rename(columns={'Month':'Date', 'RouteName':'Name', 'MonthlyRidership':'Ridership'}, inplace=True)
    # sort by Date
    df = df.sort_values('Date')
    return df

# html/3.3.a_Transit_Ridership.html
def plot_transit(df):
    trendline(
        df,
        path_html="html/3.3.a_Transit_Ridership.html",
        div_id="3.3.a_Transit_Ridership",
        x="Date",
        y="Ridership",
        color="Name",
        color_sequence=["#023f64", "#7ebfb5", "#a48352", "#FC9A62"],
        sort="Date",
        orders=None,
        x_title="Date",
        y_title="Ridership",
        markers=True,
        hover_data=None,
        tickvals=None,
        ticktext=None,
        tickangle=None,
        hovermode="x unified",
        format=",.0f",
        custom_data=["Name"],
        hovertemplate="<br>".join([
            "<b>%{y:,.0f}</b> riders on",
            "<i>%{customdata[0]}</i> lines"
                ])+"<extra></extra>",
        additional_formatting = dict(
                                    title = "Transit Ridership",
                                    margin=dict(t=20),
                                    legend=dict(
                                        # title="Transit Ridership",
                                        orientation="h",
                                        entrywidth=120,
                                        yanchor="bottom",
                                        y=1.05,
                                        xanchor="right",
                                        x=0.95,
                                    ))
    )